In [ ]:
#!/usr/bin/env python
import os
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import *
from PhysicsTools.NanoAODTools.postprocessing.trigger.triggerFilter import triggerFilter

from PhysicsTools.NanoAODTools.postprocessing.modules.common.countHistogramsModule import countHistogramsProducer
from PhysicsTools.NanoAODTools.postprocessing.framework.crabhelper import runsAndLumis


In [ ]:
##
## python helper functions
##

# allows modules to be reloaded automatically without restarting kernel
%load_ext autoreload
%autoreload 2
# widens cells
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# define some parameters for running code
isMC = True
base_path = '/eos/cms/store/group/phys_exotica/bffZprime/private_samples/signal/2016_NanoAODv6/BFFZprimeToMuMu_M175p0_dbs_0p04_2016/'
infile = '{}BFFZprimeToMuMu_M175p0_dbs_0p04_2016.root'.format(base_path)
outfile = '.'
# set maxEntries = -1 to process all events
maxEntries = 999
dataYear = '2016'
runPeriod = ''
triggers = ['HLT_Mu50', 'HLT_TkMu50', 'HLT_DoubleEle33_CaloIdL_MW', 'HLT_DoubleEle33_CaloIdL_GsfTrkIdVL', 'HLT_test']
btagWP = 0.3093 
btag_type = 'deepflavour'
selector = 'bffv2'
keep_and_drop = 'keep_and_drop_bff.txt'
crab = False

In [ ]:
#select right module
from PhysicsTools.NanoAODTools.postprocessing.modules.common.heepV72018PromptProducer import heepV72018PromptProducer
if isMC:
    from PhysicsTools.NanoAODTools.postprocessing.modules.jme.jetmetHelperRun2 import createJMECorrector
    if dataYear=='2016':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2016 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2016 as puWeight
        from PhysicsTools.NanoAODTools.postprocessing.modules.btv.btagSFProducer import btagSF2016 as btagSF
    if dataYear=='2017':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2017 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2017 as puWeight
        from PhysicsTools.NanoAODTools.postprocessing.modules.btv.btagSFProducer import btagSF2017 as btagSF
    if dataYear=='2018':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2018 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2018 as puWeight
        from PhysicsTools.NanoAODTools.postprocessing.modules.btv.btagSFProducer import btagSF2018 as btagSF
if dataYear=='2016':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2016 as muonScaleRes
if dataYear=='2017':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2017 as muonScaleRes
if dataYear=='2018':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2018 as muonScaleRes

#from PhysicsTools.NanoAODTools.postprocessing.modules.bff.WeightsProducer import WeightsProducer
    
#different preselection producerts
if selector=='inclusive':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffInclusive_preselectionModule import bffInclusivePreselProducer as preselectorProducer
if selector=='bff':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffPreselModule import bffPreselProducer as preselectorProducer
if selector=='bffv2':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffPreselModuleV2 import bffPreselProducer as preselectorProducer
if selector=='bff_eff':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffBtagEff import bffBtagEffProducer as preselectorProducer    
if selector=='minseok':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffPreselModule_minseok import bffPreselProducer as preselectorProducer

#prepare file if glob
if not crab:
    print("using input file", infile)
    import glob
    infile = glob.glob(infile)
else:
    from PhysicsTools.NanoAODTools.postprocessing.framework.crabhelper import inputFiles
    print("get input file (crab)")
    infile = inputFiles()
print("infile", infile)
#2017 EE fix
#set post fix for mc to be T1Smear
metBranchPostFix = '_T1'
if isMC:
    metBranchPostFix = '_T1Smear'
conditions_dict = {
    '2016':{'metBranchName':'MET', 'heepBranchName': 'cutBased_HEEP', 'metBranchPostFix': metBranchPostFix},
    '2017':{'metBranchName':'METFixEE2017', 'heepBranchName': 'cutBased_HEEP', 'metBranchPostFix': metBranchPostFix},
    '2018':{'metBranchName':'MET', 'heepBranchName': 'cutBased_HEEPV7p0_2018Prompt', 'metBranchPostFix': metBranchPostFix}
}




In [ ]:
#set up process for mc and data
if isMC:
    applyHEMfix=True
    jmeCorrections = createJMECorrector(
            isMC=isMC,
            dataYear=dataYear,
            runPeriod=runPeriod,
            jesUncert="Total",
            metBranchName=conditions_dict[dataYear]['metBranchName'],
            applySmearing=True,
            jetType="AK4PFchs",
            noGroom=False,
            applyHEMfix=applyHEMfix,
        )
    
    #don't use all modules for bff efficency calculation
    if 'eff' not in selector:
        modules=[
                countHistogramsProducer(),
                triggerFilter(triggers),
                btagSF(),
                jmeCorrections(),
                puWeight(),
                muonScaleRes(),
                lepSF(),
                heepV72018PromptProducer(),
                preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, applyHEMfix=applyHEMfix, **conditions_dict[dataYear])       
        ]
        
        if selector=='bffv2':
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, muon_pt = "correctedUp_pt", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, muon_pt = "correctedDown_pt", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jerDown", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jerUp", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jesTotalDown", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jesTotalUp", **conditions_dict[dataYear]))
            if applyHEMfix:
                    modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jesHEMIssueDown", **conditions_dict[dataYear]))
                    modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jesHEMIssueUp", **conditions_dict[dataYear]))
    else:
            modules=[
                btagSF(dataYear, algo=btag_type),
                jmeCorrections(),
                preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, **conditions_dict[dataYear])
            ]
    p = PostProcessor(outfile,
            infile,
            modules=modules,
            provenance=True,
            fwkJobReport=True,
            outputbranchsel=keep_and_drop,
            maxEntries=maxEntries
        )
else:
    conditions_dict[dataYear]['metBranchPostFix'] = ''
    modules=[
            countHistogramsProducer(),
            triggerFilter(triggers),
            muonScaleRes(),
            heepV72018PromptProducer(),
            preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, muon_pt = "pt", **conditions_dict[dataYear])
        ]

    p = PostProcessor(outfile,
            infile,
            modules=modules,
            provenance=True,
            fwkJobReport=True,
            outputbranchsel=keep_and_drop,
            jsonInput=runsAndLumis(),
            maxEntries=maxEntries
        )

modules

In [ ]:
import time
seconds = time.time()
local_time = time.ctime(seconds)
print("Local time:", local_time)

In [ ]:
infile

In [ ]:
p.run()

In [ ]:
import uproot as upr

In [ ]:
upf = upr.open('BFFZprimeToMuMu_M175p0_dbs_0p04_2016_Skim.root')['Events']

In [ ]:
upf.arrays([x for x in upf.keys() if 'SR1' in x], library='pd').mean(axis=0)

In [ ]:
df = upf.arrays(['Muon_effSF_trigger',
           'Muon_effSF_sys_triggerUp',
           'Muon_effSF_sys_triggerDown', 'SR1_jet_nom_muon_corrected_pt_ele_pt',
                'SR2_jet_nom_muon_corrected_pt_ele_pt'], library='pd')

In [ ]:
df.mean()

In [ ]:
upf.arrays(['SR2_jet_nom_muon_corrected_pt_ele_pt'], library='pd').sum()

In [ ]:
df[df.SR1_jet_nom_muon_corrected_pt_ele_pt==1].max(), df[df.SR2_jet_nom_muon_corrected_pt_ele_pt==1].max()

In [ ]:
df[df.SR1_jet_nom_muon_corrected_pt_ele_pt==1].min(), df[df.SR2_jet_nom_muon_corrected_pt_ele_pt==1].min()

In [ ]:
upf.arrays([ 'Muon_effSF_trigger',
 'Muon_effSF_stat_trigger',
 'Muon_effSF_ID',
 'Muon_effSF_stat_ID',
 'Muon_effSF_ISO',
 'Muon_effSF_stat_ISO',
 'Muon_effSF_sys_triggerUp',
 'Muon_effSF_sys_triggerDown',
 'Muon_effSF_sys_ID',
 'Muon_effSF_sys_ISO',], library='pd').mean()

In [ ]:
upf.arrays([ 'Muon_effSF_trigger',
 'Muon_effSF_stat_trigger',
 'Muon_effSF_ID',
 'Muon_effSF_stat_ID',
 'Muon_effSF_ISO',
 'Muon_effSF_stat_ISO',
 'Muon_effSF_sys_triggerUp',
 'Muon_effSF_sys_triggerDown',
 'Muon_effSF_sys_ID',
 'Muon_effSF_sys_ISO',], library='pd').max()*100

In [ ]:
[x for x in upf.keys() if 'SR1' in x]

In [ ]:
file_sf = "../python/postprocessing/data/leptonSF/muon_legacy/OutFile-v20190510-Combined-Run2016BtoH_Run2017BtoF_Run2018AtoD-M120to10000.root"

In [ ]:
upf2 = upr.open(file_sf)

In [ ]:
upf2.keys()

In [ ]:
import ROOT as r
import root_numpy
import numpy as np

In [ ]:
tf = r.TFile(file_sf)

In [ ]:
list(tf.GetListOfKeys())

In [ ]:
c1 = r.TCanvas()

In [ ]:
def average(*x):
    return sum(x)/len(x)

def reduce_list_by_pair(arr,func = average):
    return np.array([func(arr[i], arr[i+1]) for i in range(len(arr)-1)])

In [ ]:
th2f = tf.Get('SF_2016_errorUpper')
th2f.Draw('Colz')
c1.Draw()

array, axis = root_numpy.hist2array(th2f, return_edges=True)
array[reduce_list_by_pair(axis[0], func=max)>53]

In [ ]:
array

In [ ]:
th2f = tf.Get('SF_2016_errorUpper')
th2f.Draw('Colz')
c1.Draw()

array, axis = root_numpy.hist2array(th2f, return_edges=True)
array[reduce_list_by_pair(axis[0], func=max)>53]*100